In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install emoji
! pip install ekphrasis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 8.0 MB/s eta 0:00:00


In [ ]:
import re
import nltk
import emoji
import pandas as pd
from itertools import groupby

### Read Datasets (OLD)

In [ ]:
# Specify the path to your CSV file
csv_file = '/content/drive/Shareddrives/HLT/dataset/development/training_textual.csv'
dev_data = pd.read_csv(csv_file)
dev_data.head()

,anonymized_tweet_id,anonymized_text,label,dataset
0,217874450618134,@unknown Con tutte le denunce che si sta becca...,1,training_politics
1,360042217507605,#PRESCRIZIONE : I (t)re magi hanno trovato l’a...,0,training_politics
2,817917484817935,Il #m5s ha votato contro le #unionicivili addu...,1,training_politics
3,172580609652325,La #Lega e il #m5s stanno dando a bere all'opi...,0,training_politics
4,145836038456701,Che cosa cambia questa #legge? Caro @667517147...,0,training_politics


In [ ]:
csv_file = '/content/drive/Shareddrives/HLT/dataset/gold/test_textual_gold.csv'
test_textual_gold = pd.read_csv(csv_file)
test_textual_gold.head()

,anonymized_tweet_id,anonymized_text,label,dataset
0,424801448454884,questo anno #babbonatale portato giocattolo it...,1,test_politics
1,775194088981616,Adesso che in mezzo alla strada grazie al verg...,1,test_politics
2,386928936347190,Circondatevi di persone che non sono diventate...,0,test_politics
3,957546674360002,"Seriamente, per capire, se un ladro mi entra i...",0,test_politics
4,530526299555950,Che poi è probabile che #Spataro non sia potut...,0,test_politics


In [ ]:
tweet_corpus = pd.concat([dev_data,test_textual_gold])
tweet_corpus = tweet_corpus['anonymized_text']

In [ ]:
tweet_corpus.info()

<class 'pandas.core.series.Series'>
Int64Index: 10000 entries, 0 to 4399
Series name: anonymized_text
Non-Null Count  Dtype 
--------------  ----- 
10000 non-null  object
dtypes: object(1)
memory usage: 156.2+ KB


### Load italian corpus

In [ ]:
with open('/content/drive/Shareddrives/HLT/dataset/10M_HD3_concat_corpus.txt') as f:
    tweet_corpus = f.readlines()

In [ ]:
tweet_corpus = pd.Series(tweet_corpus)
tweet_corpus

0           #cyberattacks e #vaccinoanticovid, compromessi...
1           @user: finalmente un cambio di passo. 500mila ...
2           @user_essere: avv. mauro sandri ore 18,30, dir...
3           @user_it: 🗞️📌ansa/focus: 🗣️favo: "le vaccinazi...
4           @user: cominciamo ad usare i social anche a no...
                                  ...                        
10013610    [URL] @209567794423061 @90055554325752 #cina #...
10013611    @401560935985106 @507718538254873 @28429789767...
10013612    Trasporto pubblico, Simeone nominato consiglie...
10013613                             ma chi cazzo è letta? \n
10013614                                                [URL]
Length: 10013615, dtype: object

### Italian corpus preprocessing

In [ ]:
correct_words = ['alcool', 'cool', 'preesame']

def clean_disguised_bad_words(text):
    text = " " + text + " "
    #print(text)
    text = re.sub(r' aff[xx]+lo ', ' affanculo ', text)
    text = re.sub(r' affan[xx**]+lo ', ' affanculo ', text)
    text = re.sub(r' c[glo.x*@%#$^]+i ', ' coglioni ', text)
    text = re.sub(r' c[glo.x*@%#$^]+e ', ' coglione ', text)
    text = re.sub(r' ca\*\*o ', ' cazzo ', text, flags=re.IGNORECASE)
    text = re.sub(r' c[az.x*@%#$^]+ro ', ' cazzaro ', text)
    text = re.sub(r' inc[az.x*@%#$^]+to ', ' incazzato ', text)
    text = re.sub(r' c[az.x*@%#$^]+te ', ' cazzate ', text)
    text = re.sub(r' c[az.x*@%#$^]+i ', ' cazzi ', text)
    text = re.sub(r' c[az.x*@%#$^]+o ', ' cazzo ', text)
    text = re.sub(r' cazxxx ', ' cazzate ', text)
    text = re.sub(r' CAZO ', ' CAZZO ', text)
    text = re.sub(r' c[az.x*@%#$^]+one ', ' cazzone ', text)
    text = re.sub(r' m\*\*\*a ', ' merda ', text, flags=re.IGNORECASE)
    text = re.sub(r' m[erd.x*@%#$^]+@ ', ' merda ', text)
    text = re.sub(r' m[erd.x*@%#$^]+e ', ' merde ', text)
    text = re.sub(r' c[.x*@%#$^]+lo ', ' culo ', text)
    text = re.sub(r' culx ', ' culo ', text)
    text = re.sub(r' p[u.x*@%#$^]+ana ', ' puttana ', text)
    text = re.sub(r' p[tu.x*@%#$^]+e ', ' puttane ', text)
    text = re.sub(r' t[.x*@%#$^]+a ', ' troia ', text)
    text = re.sub(r' t[.x*@%#$^]+e ', ' troie ', text)
    text = re.sub(r' s[.x*@%#$^]+o ', ' stronzo ', text)
    text = re.sub(r' stronz[.x*@%#$^]+a ', ' stronzata ', text)
    text = re.sub(r' s[.x*@%#$^]+i ', ' stronzi ', text)
    text = re.sub(r' min[*t]+a ', ' minchiata ', text)
    return text

def deleteDuplicate (riga):
    parole = riga.split()
    stringa = ""
    for a in parole:
        parola = a
        a = [list(g) for k, g in groupby(a)]
        vocali = ['a','e','i','o','u','y']

        for idx,_ in enumerate(a):
            if idx == 0:
                stringa += a[idx][0]
            elif idx == len(a)-1:
                stringa += a[idx][0]
            elif a[idx][0] in vocali and (parola not in correct_words):
                stringa += a[idx][0]
            elif len(a[idx]) == 1:
                stringa += a[idx][0]
            elif len (a[idx]) >= 2:
                stringa += a[idx][0]
                stringa += a[idx][1]
        stringa =  stringa + " "
    return(stringa)

def preprocess_tweet(tweet):
    # Remove newlines and carriage returns
    tweet = tweet.replace('\n', ' ').replace('\r', ' ')

    # Remove incorrect bad word
    tweet = clean_disguised_bad_words(tweet)

    # Remove duplicates
    tweet = deleteDuplicate(tweet)

    # Substitute URLs with <URL>
    tweet = re.sub(r'\[URL\]', '', tweet)

    # Substitute mentions with <MENTION>
    tweet = re.sub(r'@\w+', '', tweet)

    # Substitute hashtags with <HASHTAG>
    tweet = re.sub(r'#\w+', '', tweet)

    # Replace emojis with text
    tweet = emoji.demojize(tweet, language='it')

    # Remove special characters and numbers, preserve placeholders
    processed_tweet = re.sub(r'[^a-zA-ZÀ-ú\d\U00010000-\U0010ffff]', ' ', tweet)

    return processed_tweet


In [ ]:
new_tweet_corpus = tweet_corpus[:10]
new_tweet_corpus = new_tweet_corpus.apply(preprocess_tweet)
#new_tweet_corpus.to_csv('/content/drive/Shareddrives/HLT/dataset/italian_corpus.txt', header=False, sep='\t', index=False)

In [ ]:
# Assuming new_tweet_corpus is a list of tweets or strings
for i in range(len(new_tweet_corpus)):
    print(new_tweet_corpus[i])


 e   compromessi i dati sul vaccino   l attacco è stato effettuato per fornire elementi utili ad altri paesi o aziende impegnate a sviluppare una cura contro il    indice abbassato carnagione abbastanza chiara leggi di più    
  finalmente un cambio di passo  500mila vaccinazioni al giorno e copertura dell 80  entro la fine dell estate  con il   
  avv  mauro sandri ore 18 30  diretta su visione tv  fermare il gren pas ed abolire le autorizzazioni alla commercializzazi  
   quotidiano arrotolato  puntina ansa focus   persona che parla favo   le vaccinazioni per chi ha un  sono state avviate solo in pochissime regioni  con differenze no  
  cominciamo ad usare i social anche a nostro vantaggio  altro sondaggio tra amici  siete favorevoli alla vaccinazio  
  vi fareste operare da una persona senza una laurea in medicina  no  e allora mi spiegate perché per virus e vaccini non  
  a  un operatrice socio assistenziale di una casa di riposo non si è vaccinata  è risultata positiva al  e si 

### Salvo le statistiche


In [ ]:
! python /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/generate_stats.py --input /content/drive/Shareddrives/HLT/dataset/italian_corpus.txt --name tweet_ita --ngrams 2 --mincount 70 30


computing statistics for file:  /content/drive/Shareddrives/HLT/dataset/italian_corpus.txt
100% 10013615/10013615 [07:12<00:00, 23128.87it/s]

Writing 1-grams...
entries:326,541	-	tokens:174,118,463
writing stats to file /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/../stats/tweet_ita/counts_1grams.txt
Writing 2-grams...
entries:5,653,502	-	tokens:174,118,463
writing stats to file /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/../stats/tweet_ita/counts_2grams.txt


### Test

In [ ]:
from ekphrasis.classes.segmenter import Segmenter
seg = Segmenter(corpus="/usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita")
print(seg.segment("salvinisciacallo"))
print(seg.segment("melonifuoridaicoglioni"))
print(seg.segment("salvinisciacallo"))
print(seg.segment("melonipiaga"))
print(seg.segment("salvinipagliaccio"))
print(seg.segment("governodeipeggiori"))
print(seg.segment("speranzadimettiti"))
print(seg.segment("speranzavattene"))
print(seg.segment("m5s"))
print(seg.segment("salviniportasfiga"))
print(seg.segment("unINSANOvaffanculo"))
print(seg.segment("SPERANZAVAFFANCULO"))
print(seg.segment("mavaffanculo"))
print(seg.segment("salvinimmerda"))
print(seg.segment("salvinidimettiti"))
print(seg.segment("facciamorete"))
print(seg.segment("salvinibestia"))

Reading /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita - 1grams ...
Reading /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita - 2grams ...
salvini sciacallo
meloni fuori dai coglioni
salvini sciacallo
meloni piaga
salvini pagliaccio
governo dei peggiori
speranza dimettiti
speranza vattene
m5s
salvini porta sfiga
un insan ovaffanculo
speranzavaffanculo
ma vaffanculo
salvini m merda
salvini dimettiti
facciamorete
draghi stronzo


In [ ]:
from ekphrasis.classes.segmenter import Segmenter
seg = Segmenter(corpus="twitter_2018")
print(seg.segment("salvinisciacallo"))

Reading twitter_2018 - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter_2018/counts_1grams.txt
Reading twitter_2018 - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter_2018/counts_2grams.txt
salvini sci a call o
